#### Simple Model (MNIST, CNN)
#### Ted, Jung
#### Created: 17, Jul 2025
#### Description
####   1. prepare data
####   2. split data into two(test, train)
####   3. Data scaling
####   4. Image downloads for service (tensorflow) after saving a model into directory (saved_model)
####      > docker run -t --rm -p 8501:8501 -v "./saved_model:/models/test_model" -e MODEL_NAME=test_model tensorflow/serving &

In [13]:
import tensorflow as tf
from tensorflow import keras
import os
import random
import json

fashion_mnist = keras.datasets.fashion_mnist
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

# scale the values to 0.0 to 1.0
test_images = test_images / 255.0

# reshape for feeding into the model
test_images = test_images.reshape(test_images.shape[0], 28, 28, 1)

class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

print('test_images.shape: {}, of {}'.format(test_images.shape, test_images.dtype))

rando = random.randint(0, len(test_images)-1)

data = json.dumps({"signature_name": "serving_default", "instances": test_images[0:3].tolist()})
print('Data: {} ... {}'.format(data[:50], data[len(data)-52:]))

test_images.shape: (10000, 28, 28, 1), of float64
Data: {"signature_name": "serving_default", "instances": ...  [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0]]]]}


In [ ]:
# tensorflow/serving(from Google) docker image를 실행하여 rest api구축

> docker run -t --rm -p 8501:8501 -v "./saved_model:/models/test_model" -e MODEL_NAME=test_model tensorflow/serving &

#### 예측수행

In [ ]:
import requests

headers = {"content-type": "application/json"}
json_response = requests.post('http://localhost:8501/v1/models/test_model:predict', data=data, headers=headers)
predictions = json.loads(json_response.text)['predictions']

#### 예측값 실제값 비교 (데이터 랜덤선택)

In [15]:
import np
for i in range(3):
    print(('The model predicts this was a {} (class {}), and it was actually a {} (class {})'.format(
      class_names[np.argmax(predictions[i])], np.argmax(predictions[i]), class_names[test_labels[i]], test_labels[i])))

The model predicts this was a Ankle boot (class 9), and it was actually a Ankle boot (class 9)
The model predicts this was a Pullover (class 2), and it was actually a Pullover (class 2)
The model predicts this was a Trouser (class 1), and it was actually a Trouser (class 1)
